In [1]:
import re
import spacy
import nltk
import requests
import sklearn_crfsuite
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from datasets import load_dataset
from sklearn_crfsuite import metrics
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF

C:\Users\rafro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rafro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafro\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rafro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# === 1. TEXT CLEANING & PREPROCESSING ===

In [3]:
def clean_text(text):
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"[^a-zA-Z0-9 ]", "", text)
    text = text.lower()
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(w) for w in words]
    return " ".join(words)

In [4]:
raw_text = "<p>Apple was founded by Steve Jobs.</p>"
cleaned_text = clean_text(raw_text)
print("Cleaned Text:", cleaned_text)

Cleaned Text: apple founded steve job


# === 2. NAMED ENTITY RECOGNITION (NER) ===

In [ ]:
# Load the CoNLL-2003 dataset 
dataset = load_dataset("conll2003", trust_remote_code=True) 
# Access the training, validation, and test sets 
train_dataset = dataset['train'] 
validation_dataset = dataset['validation'] 
test_dataset = dataset['test'] 
# Example: Print the first example from the training set 
print(train_dataset[0]) 

{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [22]:
# Map NER tag indices to actual labels
ner_labels = dataset["train"].features["ner_tags"].feature.int2str
print(ner_labels)  

<bound method ClassLabel.int2str of ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None)>


In [23]:
# Feature Extraction for CRF Model
def extract_features(doc):
    return [{"word": w} for w in doc["tokens"]]

def get_labels(doc):
    return [ner_labels(label) for label in doc["ner_tags"]]

In [24]:
X_train = [extract_features(sent) for sent in train_dataset]
y_train = [get_labels(sent) for sent in train_dataset]
X_test = [extract_features(sent) for sent in test_dataset]
y_test = [get_labels(sent) for sent in test_dataset]

print("Sample Features:", X_train[0])
print("Sample Labels:", y_train[0])

Sample Features: [{'word': 'EU'}, {'word': 'rejects'}, {'word': 'German'}, {'word': 'call'}, {'word': 'to'}, {'word': 'boycott'}, {'word': 'British'}, {'word': 'lamb'}, {'word': '.'}]
Sample Labels: ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


In [25]:
# Train CRF model
crf = sklearn_crfsuite.CRF(
    algorithm="lbfgs",
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    max_iterations=100)

In [26]:
# Evaluate CRF model
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       B-LOC       0.89      0.78      0.83      1668
      B-MISC       0.83      0.67      0.74       702
       B-ORG       0.81      0.55      0.66      1661
       B-PER       0.74      0.43      0.55      1617
       I-LOC       0.42      0.66      0.51       257
      I-MISC       0.63      0.60      0.61       216
       I-ORG       0.53      0.63      0.58       835
       I-PER       0.72      0.55      0.62      1156
           O       0.94      0.97      0.95     38323

    accuracy                           0.91     46435
   macro avg       0.72      0.65      0.67     46435
weighted avg       0.90      0.91      0.90     46435



# === 3. RELATION EXTRACTION (RE) ===

# === 4. KNOWLEDGE GRAPH BUILDING ===